# Import library

In [1]:
import os
import pandas as pd
from hta.common.trace import Trace
from hta.configs.config import logger
from hta.analyzers.timeline import plot_timeline_gpu_kernels

from pathlib import Path

## Display figures on github

+ The following cell enables the figures visible on github but causes the figure being  non-interactive.

+ To enable interactive viewing of the figures, set `ON_GITHUB = False` when running this notebook outside github.

+ To run all cells to get all the figures.

In [2]:
# ON_GITHUB = True
ON_GITHUB = False
if ON_GITHUB:
    import plotly.io as pio
    pio.renderers.default = "svg"

# Load Trace Data

In [ ]:
%%time

# Detect the HolisticTraceAnalysis project path
project_path_candidates=!find ~/ -name HolisticTraceAnalysis
for p in project_path_candidates:
    if os.path.exists(p) and os.path.exists(Path(p).joinpath("hta")):
        base_dir = p
        break
        
# Specify a trace folder
# Ensure the sample datasets exists
assert base_dir is not None
assert os.path.exists(f"{base_dir}/tests/data/vision_transformer")

trace_dir = str(Path(base_dir).joinpath("tests/data/vision_transformer"))
logger.info(f"Using traces from folder {trace_dir}")

# Parse the traces
trace_data = Trace(trace_dir=trace_dir)
trace_data.parse_traces()

# Prepare a DataFrame that contains all the traces

In [ ]:
%%time
_ranks = list(trace_data.get_all_traces().keys())
df = pd.concat([trace_data.get_trace(r) for r in _ranks], axis=0, keys=_ranks, names=["rank", "idx"]).reset_index()
symbol_table = trace_data.symbol_table

In [5]:
df

,rank,idx,index,name,pid,tid,ts,cat,dur,stream,correlation,Trace iteration,memory_bw_gbps,index_correlation,iteration
0,0,23850,23850,110,715861,715861,1658948133801383,101,1068,-1,-1,0,-1.0,-1,-1
1,0,23851,23851,51,715861,715861,1658948133802396,101,40,-1,-1,0,-1.0,-1,-1
2,0,23852,23852,103,715861,715861,1658948133802450,101,0,-1,-1,0,-1.0,-1,-1
3,0,23853,23853,1,715861,715861,1658948133802479,101,541820,-1,-1,0,-1.0,-1,15
4,0,23854,23854,51,715861,715861,1658948133802483,101,11,-1,-1,0,-1.0,-1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993506,7,310907,310907,93,7,26,1658948134997354,229,128,26,338677,-1,-1.0,310909,17
1993507,7,310909,310909,248,715868,715868,1658948134960850,195,16,-1,338677,-1,-1.0,310907,17
1993508,7,310911,310911,41,715868,715868,1658948134960867,195,1,-1,338678,-1,-1.0,0,17
1993509,7,310912,310912,107,715868,715868,1658948134960871,195,1,-1,338681,-1,-1.0,0,17


In [6]:
new_df = df[df['rank']<4]

In [7]:
new_df

,rank,idx,index,name,pid,tid,ts,cat,dur,stream,correlation,Trace iteration,memory_bw_gbps,index_correlation,iteration
0,0,23850,23850,110,715861,715861,1658948133801383,101,1068,-1,-1,0,-1.0,-1,-1
1,0,23851,23851,51,715861,715861,1658948133802396,101,40,-1,-1,0,-1.0,-1,-1
2,0,23852,23852,103,715861,715861,1658948133802450,101,0,-1,-1,0,-1.0,-1,-1
3,0,23853,23853,1,715861,715861,1658948133802479,101,541820,-1,-1,0,-1.0,-1,15
4,0,23854,23854,51,715861,715861,1658948133802483,101,11,-1,-1,0,-1.0,-1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998215,3,316237,316237,44,715864,715864,1658948134911510,195,1,-1,338172,-1,-1.0,0,17
998216,3,316238,316238,202,715864,715864,1658948134911519,195,1,-1,338176,-1,-1.0,0,17
998217,3,316239,316239,202,715864,715864,1658948134911523,195,0,-1,338179,-1,-1.0,0,17
998218,3,316240,316240,240,715864,715864,1658948134911781,195,1,-1,338186,-1,-1.0,0,17


# Example #1 - Plot GPU kernels on all ranks for one iteration

In [ ]:
%%time
plot_timeline_gpu_kernels("Timeline of GPU Kernels (Iteration=15)", new_df, symbol_table, iterations=[15], duration_threshold=2000)

# Example #2 - Plot compute kernels for two iterations

In [ ]:
%%time
s_map = pd.Series(symbol_table.get_sym_id_map())
non_computer_name_ids = s_map[
    s_map.index.str.startswith("ncclKernel")
    | s_map.index.str.startswith("Memset")
    | s_map.index.str.startswith("Memcpy")
].values
compute_df = df.loc[~df["name"].isin(non_computer_name_ids)]
plot_timeline_gpu_kernels("Timeline of Compute Kernels (Iteration = [15, 16])", compute_df, symbol_table, iterations=[15, 16], duration_threshold=2000)

# Example #3 - Plot compute kernels on one stream for one iterations 

In [ ]:
s_map = pd.Series(symbol_table.get_sym_id_map())
non_compute_name_ids = s_map[
    s_map.index.str.startswith("ncclKernel")
    | s_map.index.str.startswith("Memset")
    | s_map.index.str.startswith("Memcpy")
].values
compute_df = df.loc[~df["name"].isin(non_compute_name_ids)]
plot_timeline_gpu_kernels("Timeline of Computer Kernels (Iteration=16, Stream=7)", compute_df, symbol_table, iterations=[16], streams=[7], duration_threshold=2000)

# Example 4 - Plot all communication kernels on all iterations

In [ ]:
%%time
s_map = pd.Series(symbol_table.get_sym_id_map())
communicate_name_ids = s_map[
    s_map.index.str.startswith("ncclKernel")
].values
communicate_df = df.loc[df["name"].isin(communicate_name_ids)]
plot_timeline_gpu_kernels("Timeline of All Communication Kernels", communicate_df, symbol_table, duration_threshold=2000)

# Example 5 - Plot all-to-all communication kernels on ranks 0, 1, 2

In [ ]:
%%time
s_map = pd.Series(symbol_table.get_sym_id_map())
communicate_name_ids = s_map[
    s_map.index.str.startswith("ncclKernel_ReduceScatter")
].values
communicate_df = df.loc[df["name"].isin(communicate_name_ids)]
plot_timeline_gpu_kernels("All-to-All Communication Kernels on Ranks [0, 1, 2]", communicate_df, symbol_table, ranks=[0, 1, 2], duration_threshold=2000)